In [21]:
# import lightgbm as lgb
# from sklearn.model_selection import KFold
# from scipy import sparse
import warnings
import time
import sys
import os
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from model_zoo import my_lgb,my_xgb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline

In [22]:
# 评分函数与提交函数
def score(pre, truth):
    return 1 / (MAE(pre, truth) + 1)

def MAE(pre, truth):
    return abs((np.rint(pre) - truth)).mean()

def submit(model_name='default', predictions=None):
    sub_df = pd.read_csv('dataset/submit_example.csv')
    sub_df[' score'] = np.rint(predictions).astype(int)
    sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [23]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

特征工程，简单粗暴。

* 长尾数据处理

* 过拟合

In [24]:
def percent(df, feature):
    for f in feature:
        data[f] = np.rint(data[f])
        new_feature = f + '_count'
        tmp = df.groupby(f).size().reset_index().rename(columns={0:new_feature})
        #  tmp[new_feature] = tmp[new_feature].cumsum()*100/df.shape[0]
        df = df.merge(tmp, 'left', on = f)
    return df
    
def generate_feature(df):
    df['用户前五个月平均消费值（元）'] = (df['用户近6个月平均消费值（元）']*6 - df['用户账单当月总费用（元）'])/5
    df['当月消费值较前五个月平均消费值'] = df['用户账单当月总费用（元）'] - df['用户前五个月平均消费值（元）']
    app_col = ['当月视频播放类应用使用次数','当月金融理财类应用使用总次数','当月网购类应用使用次数']
    df['是否使用网购类应用'] = np.where(df['当月网购类应用使用次数'] > 0, 1, 0)
    df['当月网购类应用使用次数' + '百分比'] = (df['当月网购类应用使用次数'])/(df[app_col].sum(axis=1) + 1e-8)
    df.loc[df['用户年龄']==0, '用户年龄'] = df['用户年龄'].mode()

    return df

In [25]:
target = train_df['信用分']
data = pd.concat([train_df.drop(columns=['信用分']), test_df], axis=0, ignore_index=True)
data = generate_feature(data)
train = data.loc[:49999, :]
test = data.loc[50000:, :]

In [26]:
drop_columns = ['用户编码']
X_train = train.drop(columns=drop_columns).values
y_train = target.values
X_test = test.drop(columns=drop_columns).values

param = {'num_leaves': 30,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.008,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.04,
         "verbosity": -1}

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.004,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.05,
         "verbosity": -1}
 94

阴差阳错我用的都是mse,只是用了不同的随机种子

In [50]:
param = {'num_leaves': 40,
         'objective':'regression_l2',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}
clf = my_lgb(folds=5, seed=2018)
clf.inference_folds(X_train, y_train, X_test, param)
mse_2018 = clf.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.5588	valid_1's l1: 20.805
[400]	training's l1: 16.327	valid_1's l1: 16.7958
[600]	training's l1: 14.9932	valid_1's l1: 15.6131
[800]	training's l1: 14.5348	valid_1's l1: 15.2435
[1000]	training's l1: 14.3203	valid_1's l1: 15.1045
[1200]	training's l1: 14.183	valid_1's l1: 15.0436
[1400]	training's l1: 14.0776	valid_1's l1: 15.0116
[1600]	training's l1: 13.9905	valid_1's l1: 14.992
[1800]	training's l1: 13.9112	valid_1's l1: 14.9778
[2000]	training's l1: 13.8415	valid_1's l1: 14.9654
[2200]	training's l1: 13.771	valid_1's l1: 14.9561
[2400]	training's l1: 13.7077	valid_1's l1: 14.9493
[2600]	training's l1: 13.647	valid_1's l1: 14.9439
[2800]	training's l1: 13.5881	valid_1's l1: 14.9411
[3000]	training's l1: 13.531	valid_1's l1: 14.9367
[3200]	training's l1: 13.4743	valid_1's l1: 14.9336
[3400]	training's l1: 13.4207	valid_1's l1: 14.9311
[3600]	training's l1: 13.3665	valid_1's l1: 14.9289
[38

In [49]:
param = {'num_leaves': 35,
         'objective':'regression_l2',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 10,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mse = my_lgb(folds=5, seed=2019)
clf_mse.inference_folds(X_train, y_train, X_test, param)
mse_2019 = clf_mse.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.6696	valid_1's l1: 20.731
[400]	training's l1: 16.4673	valid_1's l1: 16.584
[600]	training's l1: 15.1318	valid_1's l1: 15.3514
[800]	training's l1: 14.6669	valid_1's l1: 14.9726
[1000]	training's l1: 14.4531	valid_1's l1: 14.8317
[1200]	training's l1: 14.3198	valid_1's l1: 14.7661
[1400]	training's l1: 14.214	valid_1's l1: 14.7268
[1600]	training's l1: 14.1288	valid_1's l1: 14.7028
[1800]	training's l1: 14.0533	valid_1's l1: 14.6871
[2000]	training's l1: 13.9855	valid_1's l1: 14.6748
[2200]	training's l1: 13.9195	valid_1's l1: 14.6654
[2400]	training's l1: 13.86	valid_1's l1: 14.6583
[2600]	training's l1: 13.8024	valid_1's l1: 14.6525
[2800]	training's l1: 13.7431	valid_1's l1: 14.6481
[3000]	training's l1: 13.6881	valid_1's l1: 14.6451
[3200]	training's l1: 13.6353	valid_1's l1: 14.6432
[3400]	training's l1: 13.5832	valid_1's l1: 14.6401
[3600]	training's l1: 13.5306	valid_1's l1: 14.637
[3

In [ ]:
p = 0.502
q = 0.50
results = mse_2018*p+ mse_2019*q 
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q
# submit(model_name='model_V2_bad', predictions=res)

In [48]:
param = {'num_leaves': 35,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mae = my_lgb(folds=5, seed=2018)
clf_mae.inference_folds(X_train, y_train, X_test, param)
mae_2018 = clf_mae.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.6229	valid_1's l1: 20.8667
[400]	training's l1: 16.6054	valid_1's l1: 17.0527
[600]	training's l1: 15.1907	valid_1's l1: 15.8011
[800]	training's l1: 14.6575	valid_1's l1: 15.3725
[1000]	training's l1: 14.4027	valid_1's l1: 15.1852
[1200]	training's l1: 14.2428	valid_1's l1: 15.0938
[1400]	training's l1: 14.1222	valid_1's l1: 15.0397
[1600]	training's l1: 14.0218	valid_1's l1: 15.0082
[1800]	training's l1: 13.9324	valid_1's l1: 14.9833
[2000]	training's l1: 13.852	valid_1's l1: 14.9633
[2200]	training's l1: 13.7749	valid_1's l1: 14.9509
[2400]	training's l1: 13.7035	valid_1's l1: 14.9405
[2600]	training's l1: 13.6368	valid_1's l1: 14.9332
[2800]	training's l1: 13.5731	valid_1's l1: 14.9291
[3000]	training's l1: 13.5119	valid_1's l1: 14.9233
[3200]	training's l1: 13.453	valid_1's l1: 14.9177
[3400]	training's l1: 13.3964	valid_1's l1: 14.9143
[3600]	training's l1: 13.3411	valid_1's l1: 14.911

In [ ]:
# 0.06398976
clf_mae.submit(output_name = 'model_V1_mae_2018')

In [47]:
param = {'num_leaves': 37,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mae_ = my_lgb(folds=5, seed=2019)
clf_mae_.inference_folds(X_train, y_train, X_test, param)
mae_2019 = clf_mae_.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.6621	valid_1's l1: 20.715
[400]	training's l1: 16.6407	valid_1's l1: 16.7797
[600]	training's l1: 15.2322	valid_1's l1: 15.4893
[800]	training's l1: 14.6974	valid_1's l1: 15.0549
[1000]	training's l1: 14.4384	valid_1's l1: 14.8868
[1200]	training's l1: 14.277	valid_1's l1: 14.8123
[1400]	training's l1: 14.1564	valid_1's l1: 14.7695
[1600]	training's l1: 14.0564	valid_1's l1: 14.743
[1800]	training's l1: 13.966	valid_1's l1: 14.7235
[2000]	training's l1: 13.8856	valid_1's l1: 14.7099
[2200]	training's l1: 13.81	valid_1's l1: 14.7007
[2400]	training's l1: 13.7369	valid_1's l1: 14.6912
[2600]	training's l1: 13.6698	valid_1's l1: 14.6847
[2800]	training's l1: 13.6044	valid_1's l1: 14.6773
[3000]	training's l1: 13.5431	valid_1's l1: 14.672
[3200]	training's l1: 13.483	valid_1's l1: 14.6655
[3400]	training's l1: 13.4244	valid_1's l1: 14.6605
[3600]	training's l1: 13.3693	valid_1's l1: 14.6545
[380

In [ ]:
# 0.06399214
clf_mae_.submit(output_name = 'model_V1_mae_2019')

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mse',
         "lambda_l1": 0.16,
         "lambda_l2": 0.1,
         "verbosity": -1}

In [ ]:
p = 0.25
q = 0.25
m = 0.251
n = 0.25
results = mse_2018*p+ mse_2019*q + mae_2018*m + mae_2019*n
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q + clf_mae.results*m + clf_mae_.results*n
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
residual = mae_2018 - y_train
submit(model_name='resdiual_mae', predictions=residual)

In [ ]:
# drop_columns = drop_columns.remove('信用分')
col = train.drop(columns=drop_columns).columns
f = clf_mae.importance_feature(col)
f

In [ ]:
(residual>100).sum()

In [ ]:
feature = ['用户网龄（月）', '当月通话交往圈人数', '用户年龄', 
       '用户账单当月总费用（元）', '用户近6个月平均消费值（元）', '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数',
       '当月网购类应用使用次数', '用户当月账户余额（元）', '近三个月月均商场出现次数',
       '当月旅游资讯类应用使用次数', '缴费用户最近一次缴费金额（元）', '信用分']

In [ ]:
train_df.loc[residual > 100,feature].sample(10)

In [ ]:
train_df.loc[23379, :]